In [8]:
from sympy import *
from sympy.vector import *

Space=CoordSys3D('ViewSpace')
ColorSpace=CoordSys3D('Color')
i=Space.i; j=Space.j; k=Space.k
r=ColorSpace.i; g=ColorSpace.j; b=ColorSpace.k

def makeVector(name):
    return Symbol(name+"_x")*i + Symbol(name+"_y")*j + Symbol(name+"_z")*k
def makeColor(name):
    return Symbol(name+"_r")*r + Symbol(name+"_g")*g + Symbol(name+"_b")*b

Energy=Symbol('E')
Lpos=makeVector('L')
Lcol=makeColor('C')
View=makeVector('V')
Norm=makeVector('N')


In [9]:
def superrec(thing,func):
    out=[]
    for item in thing:
        isiter=False
        try:
            iter(item)
            isiter=True
        except:
            pass

        if(isiter):
            out = out + superrec(item,func)
        else:
            out.append(func(item))
    return out

def rec_count(thing):
    if(hasattr(thing,'count_ops')):
        return thing.count_ops()
    return 0

def compute_cost(ex):
    return sum(superrec(cse(ex),rec_count))

class CseNamer:
    name = "tmp"
    counter = 0

    def __iter__(self):
        return self
    
    def __next__(self):
        self.counter+=1
        return self.name+str(self.counter)

In [10]:
#point
Ldir=(View-Lpos).normalize()
diffuse=dot(-Ldir,Norm)
display(Eq(Symbol('diffuse'),diffuse))

Refl=Ldir-2*Ldir.projection(Norm)
specular=Max(dot(-View,Refl),0)
display(Eq(Symbol('specular'),specular))

dist=(Lpos-View).magnitude()
atten=1/(dist**2)

pointlight=(diffuse)*atten*Energy
pointlight = simplify(pointlight)
display(pointlight)
count_ops(cse(pointlight))

Eq(diffuse, -N_x*(-L_x + V_x)/sqrt((-L_x + V_x)**2 + (-L_y + V_y)**2 + (-L_z + V_z)**2) - N_y*(-L_y + V_y)/sqrt((-L_x + V_x)**2 + (-L_y + V_y)**2 + (-L_z + V_z)**2) - N_z*(-L_z + V_z)/sqrt((-L_x + V_x)**2 + (-L_y + V_y)**2 + (-L_z + V_z)**2))

Eq(specular, Max(0, -V_x*((-L_x + V_x)/sqrt((-L_x + V_x)**2 + (-L_y + V_y)**2 + (-L_z + V_z)**2) - 2*(-L_x + V_x)*(N_x*(-L_x + V_x)/sqrt((-L_x + V_x)**2 + (-L_y + V_y)**2 + (-L_z + V_z)**2) + N_y*(-L_y + V_y)/sqrt((-L_x + V_x)**2 + (-L_y + V_y)**2 + (-L_z + V_z)**2) + N_z*(-L_z + V_z)/sqrt((-L_x + V_x)**2 + (-L_y + V_y)**2 + (-L_z + V_z)**2))/(((-L_x + V_x)**2/((-L_x + V_x)**2 + (-L_y + V_y)**2 + (-L_z + V_z)**2) + (-L_y + V_y)**2/((-L_x + V_x)**2 + (-L_y + V_y)**2 + (-L_z + V_z)**2) + (-L_z + V_z)**2/((-L_x + V_x)**2 + (-L_y + V_y)**2 + (-L_z + V_z)**2))*sqrt((-L_x + V_x)**2 + (-L_y + V_y)**2 + (-L_z + V_z)**2))) - V_y*((-L_y + V_y)/sqrt((-L_x + V_x)**2 + (-L_y + V_y)**2 + (-L_z + V_z)**2) - 2*(-L_y + V_y)*(N_x*(-L_x + V_x)/sqrt((-L_x + V_x)**2 + (-L_y + V_y)**2 + (-L_z + V_z)**2) + N_y*(-L_y + V_y)/sqrt((-L_x + V_x)**2 + (-L_y + V_y)**2 + (-L_z + V_z)**2) + N_z*(-L_z + V_z)/sqrt((-L_x + V_x)**2 + (-L_y + V_y)**2 + (-L_z + V_z)**2))/(((-L_x + V_x)**2/((-L_x + V_x)**2 + (-L_y + V_y)**2

E*(N_x*(L_x - V_x) + N_y*(L_y - V_y) + N_z*(L_z - V_z))/((L_x - V_x)**2 + (L_y - V_y)**2 + (L_z - V_z)**2)**(3/2)

17

In [14]:

Integral(pointlight, Lpos.components[i])
LposA = makeVector("L_a")
LposB = makeVector("L_b")
t = symbols("t")
lerped_lpos = LposA*(1-t)+LposB*t
linelight = pointlight.subs({Lpos.components[c]:lerped_lpos.components[c] for c in [i,j,k]})
display(linelight)
linelight = integrate(linelight,t).doit()
display(linelight)

E*(N_x*(L_a_x*(1 - t) + L_b_x*t - V_x) + N_y*(L_a_y*(1 - t) + L_b_y*t - V_y) + N_z*(L_a_z*(1 - t) + L_b_z*t - V_z))/((L_a_x*(1 - t) + L_b_x*t - V_x)**2 + (L_a_y*(1 - t) + L_b_y*t - V_y)**2 + (L_a_z*(1 - t) + L_b_z*t - V_z)**2)**(3/2)

KeyboardInterrupt: 

In [ ]:
Ldir.components[i]

(-L_x + V_x)/sqrt((-L_x + V_x)**2 + (-L_y + V_y)**2 + (-L_z + V_z)**2)